In [ ]:
cd("/Users/evanrussek/foraging/")

using CSV
using DataFrames
using DataFramesMeta
using CategoricalArrays
using Gadfly
# using MixedModels
# using StatsBase
using Statistics
# import Cairo
using Distributions
using SpecialFunctions
using StatsFuns
using Optim
using ForwardDiff
using PyCall

include("sim_lag_functions.jl")
include("sim_learn_funcs.jl")

# %% markdown

hello

# %% markdown



# start reward was 120,90,60

param_dict = Dict();
param_dict["harvest_cost"] = 1.;
param_dict["travel_cost_easy"] = 3.;
param_dict["travel_cost_hard"] = 5.;
param_dict["r_hat_start_reward_weight"] = 1. / 10;
param_dict["r_hat_start_easy_weight"] = 1.;
param_dict["harvest_lag_hat_start"] = 1.0; # don't fit this...
param_dict["harvest_bias"] = 0;
param_dict["choice_beta"] = 1.;
param_dict["lag_beta"] = 5.;
param_dict["lr_R_hat_pre"] = -2.9;
param_dict["lr_harvest_lag_hat_pre"] = -2.;
transform_lr(param_dict["lr_R_hat_pre"])

param_dict

sim_df = sim_forage_learn(param_dict);

first(sim_df,6)


plot(sim_df, x = :time, y = :reward_obs, xgroup = :travel_key_cond,ygroup = :start_reward, color = :start_reward,
    Geom.subplot_grid(Geom.line))

plot(sim_df, x = :time, y = :R_hat, xgroup = :travel_key_cond,
    group = :trial_num, color = :start_reward, linestyle = :travel_key_cond,
    Geom.subplot_grid(Geom.line))

plot(sim_df, x = :time, y = :harvest_lag_hat, group = :trial_num, color = :start_reward,
 linestyle = :travel_key_cond, Geom.line)

plot(sim_df, x = :time, y = :threshold, group = :trial_num, color = :start_reward,
    linestyle = :travel_key_cond, Geom.line)

### plot the exit threshold...
make_exit_plot(sim_df)

make_lag_plot(sim_df)


########## likelihood function
param_names = [];
param_vals = Float64[];
for (k,v) in param_dict
    #println(k,v)
    push!(param_names, k)
    push!(param_vals, v)
end
print(param_names) # check that this matches the order in the likelihood function...
include("sim_learn_funcs.jl")

# check how long different versions of like fun take
@time nll1 = forage_learn_lik(param_vals, sim_df)

@time nll2 = forage_learn_lik2(param_vals, sim_df)

# check that function runs at start params
nparam = length(param_dict)
start_x = zeros(nparam);

include("sim_learn_funcs.jl")
forage_learn_lik2(start_x, sim_df) # why is this a nan...
# this can't be

function relu_sm(b)
    # makes minimum value .0001
	p=1/(1+exp(-100*(.0001 -b)))
	return(p * .0001 + (1-p) * b)
end


# softmaximum function plot
plot(relu_sm, -100., 40., Guide.ylabel("Soft Maximum"), Guide.xlabel("Input"))


#
cost_fun = (x) -> forage_learn_lik2(x,sim_df);

bm = 3.;
sp = (x) -> log(1. + exp(x))/bm + .0001
sp2 = (x) -> x*erf(1.702*x)

erf(-1)

?erf

C = 0;

C_new = C < .0001 ? .001 : C

plot([sp2, (x) -> x, (x) -> erf(x)], -100, 100)

include("sim_learn_funcs.jl")
ForwardDiff.gradient(cost_fun,start_x)

# o use python's scipy.optimize...
#so = pyimport("scipy.optimize")
#@time a = so.minimize(cost_fun, start_x, method="L-BFGS-B", jac = (x->ForwardDiff.gradient(cost_fun,x)))


# julia optimize -- should be faster
a = optimize((x) -> forage_learn_lik2(x,sim_df), start_x, LBFGS(); autodiff=:forward)





a.minimum

p_hat = a.minimizer

# to dict...
p_hat_dict = Dict()
for j in 1:length(param_names)
    p_hat_dict[param_names[j]] = p_hat[j]
end

sim_df_rec = sim_forage_learn(p_hat_dict);
p_rec_choice = make_exit_plot(sim_df_rec)
p_orig_choice = make_exit_plot(data)
vstack([p_orig_choice; p_rec_choice])


p_rec_lag = make_lag_plot(sim_df_rec)
p_orig_lag = make_lag_plot(data)
vstack([p_orig_lag; p_rec_lag])


# is current param like better than orig param lik?
new_lik = forage_learn_lik(p_hat,sim_df)
# original values much worse...

include("sim_learn_funcs.jl")
orig_lik = forage_learn_lik(param_vals,sim_df)

harvest_df = @where(sim_df, :phase .== "HARVEST")
findall(harvest_df[:choice] .== 2)

# make a dictionary with simulated parameter value and recovered parameter value...

original_val = zeros(length(param_dict));
recovered_val = zeros(length(param_dict));
param_name = [];
i = 1;
for (j,v) in param_dict
    println(j)
    push!(param_name,j);
    original_val[i] = v;
    recovered_val[i] = p_hat_dict[j]
    global i = i+1;
    #println(j,v)
end
fit_df = DataFrame(pname = param_name, original_val = original_val, recovered_val = recovered_val)


params= start_x
data = sim_df;